In [26]:
import requests
import pandas as pd
from tqdm.notebook import tqdm
import time, random

### Lấy category id

In [27]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi,en-US;q=0.9,en;q=0.8',
    'x-guest-token': 'acuqSeZsnKCWVgofFGjxJi7Ot4N8dPpX',
    'Connection': 'keep-alive',
}

In [28]:
response = requests.get("https://api.tiki.vn/raiden/v2/menu-config", headers=HEADERS)

categories_id = []
if response.status_code == 200:
    menu = response.json().get('menu_block').get('items')
    categories_id = [category['link'].split('/c')[-1] for category in menu]

time.sleep(random.uniform(0.5, 2))
print(len(categories_id))

26


### Lấy id của sản phẩm

In [29]:
products_id = []

for category_id in tqdm(categories_id):
    response = requests.get(f'https://tiki.vn/api/personalish/v1/blocks/listings?category={category_id}', headers=HEADERS)
    
    if response.status_code == 200:
        for product in response.json().get('data'):
            products_id.append(product.get('id'))
    else:
        print(f'Category {category_id}: {response}')
    
    time.sleep(random.uniform(0.5, 2))
print(len(products_id))

  0%|          | 0/26 [00:00<?, ?it/s]

1040


### Lấy comment trên từng sản phẩm

In [ ]:
def comment_parser(json):
    dic = dict()
    dic['id'] = json.get('id', None)
    dic['title'] = json.get('title', None)
    dic['content'] = json.get('content', None)
    dic['rating'] = json.get('rating', None)
    dic['created_at'] = json.get('created_at', None)
    dic['purchased_at'] = json.get('created_by', {}).get('purchased_at', None)
    return dic

params = {
    'limit': 5,
    'page': 1,
    'include': 'comments',
    'sort': 'score|desc,id|desc,stars|all'
}

comments = []
for product_id in tqdm(products_id):
    params['product_id'] = product_id
    response = requests.get('https://tiki.vn/api/v2/reviews', headers=HEADERS, params=params)

    if response.status_code == 200:
        for comment in response.json().get('data'):
            comments.append(comment_parser(comment))
    else:
        print(f'Product {product_id}: {response.status_code}')

    time.sleep(random.uniform(0.5, 2))
print(len(comments))

  0%|          | 0/1040 [00:00<?, ?it/s]

4707


### Lấy thông tin sản phẩm

In [33]:
def parser_product(json):
    dic = dict()
    dic['id'] = json.get('id', None)
    dic['name'] = json.get('name', None)
    dic['short_description'] = json.get('short_description', None)
    dic['price'] = json.get('price', None)
    dic['list_price'] = json.get('list_price', None)
    dic['discount'] = json.get('discount', None)
    dic['discount_rate'] = json.get('discount_rate', None)
    dic['all_time_quantity_sold'] = json.get('all_time_quantity_sold', None)
    dic['inventory_status'] = json.get('inventory_status', None)
    dic['stock_item_qty'] = json.get('stock_item', {}).get('qty', None)
    dic['stock_item_max_sale_qty'] = json.get('stock_item', {}).get('max_sale_qty', None)
    return dic

product_details = []
for product_id in tqdm(products_id):
    response = requests.get(f'https://tiki.vn/api/v2/products/{product_id}', headers=HEADERS)

    if response.status_code == 200:
        product_details.append(parser_product(response.json()))
    else:
        print(f'Product {product_id}: {response.status_code}')
    
    time.sleep(random.uniform(0.5, 2))

print(len(product_details))

  0%|          | 0/1040 [00:00<?, ?it/s]

1040


In [34]:
df_product = pd.DataFrame(product_details)
df_comment = pd.DataFrame(comments)

In [ ]:
df_product.to_parquet()